In [2]:
#Import Packages

import torch
import tqdm as notebook_tqdm
import json
import csv

import gensim
from gensim.models import Word2Vec

import numpy as np
from numpy import array  
from numpy.linalg import norm

In [6]:
#Open Dataset
file =  open("SemEvalData/2013/test-gs/STS.input.OnWN.txt", "r", encoding='utf-8')
file_temp = file.readlines()
training_raw_list = []
for line in file_temp:
    training_raw_list.append(line.split('\t'))

In [7]:
#Open Label File
file =  open("SemEvalData/2013/test-gs/STS.gs.OnWN.txt", "r", encoding='utf-8')
file_temp = file.readlines()
training_raw_labels = []
for line in file_temp:
    training_raw_labels.append(float(line.strip()))

In [8]:
#Initial processing - Split sentence pairs

training_list1 = []
training_list2 = []
for i in range(len(training_raw_list)):        
    temp_sent1 = training_raw_list[i][0]
    temp_sent2 = training_raw_list[i][1]
    temp_sent2 = temp_sent2.replace("\n", "")
    training_list1.append(temp_sent1)
    training_list2.append(temp_sent2)


In [9]:
print(len(training_list1))
print(len(training_list2))
print(len(training_raw_labels))

561
561
561


In [10]:
#Save data to 2 text files for CoreNLP parsing

file1 = open("STS-2013_OnWN1.txt", "w")
file2 = open("STS-2013_OnWN2.txt", "w")
for element in training_list1:
    file1.write('%s\n' % element)
count = 0
for element in training_list2:
    file2.write('%s\n' % element)
    count += 1
    file1.close()
file2.close()

In [11]:
file1 = open("STS-2013_OnWN1.txt", "r")
file2 = open("STS-2013_OnWN2.txt", "r")
temp1 = file1.readlines()
temp2 = file2.readlines()
print(len(temp1))
print(len(temp2))
file1.close()
file2.close()

561
561


In [18]:
#Process Sentence Text File with CoreNLP 
#java -cp "*" -Xmx8g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators "tokenize,ssplit,pos,lemma,ner" -tokenize.whitespace=true -ssplit.eolonly -file training_list2.txt -outputFormat json

In [12]:
#Load CoreNLP Output JSONs

with open("STS-2013_OnWN1.json", "r") as read_file1:
    training_json1 = json.load(read_file1)
with open("STS-2013_OnWN2.json", "r") as read_file2:
    training_json2 = json.load(read_file2)
read_file1.close()
read_file2.close()

In [13]:
print(len(training_json1['sentences']))
print(len(training_json2['sentences']))

561
561


In [14]:
core_words_train1 = []
core_words_train2 = []
max_sen_length_train = 0

In [15]:
#Create SPO core words lists based on CoreNLP parsing

for i in range(len(training_json1['sentences'])):
    temp_list = []
    for j in training_json1['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_train1.append(temp_list)
    if len(temp_list) > max_sen_length_train:
        max_sen_length_train = len(temp_list)
        
for i in range(len(training_json2['sentences'])):
    temp_list = []
    for j in training_json2['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_train2.append(temp_list)
    if len(temp_list) > max_sen_length_train:
        max_sen_length_train = len(temp_list)
        

In [16]:
print(len(core_words_train1))
print(len(core_words_train2))
print(max_sen_length_train)

561
561
10


In [19]:
#Combine SPO core words to create training list for Word2Vec
 
word2vec_masterlist = []
for line in core_words_train1:
    word2vec_masterlist.append(line)
for line in core_words_train2:
    word2vec_masterlist.append(line)

In [20]:
len(word2vec_masterlist)

1122

In [21]:
#Train Word2Vec model
wv_model = gensim.models.Word2Vec(word2vec_masterlist, vector_size=50, workers=4, min_count=1)

In [22]:
#Divide data into training and test sets (80/20)
 
x = len(core_words_train1)
x_tr_end = int(0.8*x)
training_list_80_1 = core_words_train1[0:x_tr_end]
training_list_80_2 = core_words_train2[0:x_tr_end]

test_list_20_1 = core_words_train1[x_tr_end:x]
test_list_20_2 = core_words_train2[x_tr_end:x]


In [23]:
print(len(training_list_80_1))
print(len(test_list_20_1))

448
113


In [24]:
train_tensors1 = []
train_tensors2 = []
test_tensors1 = []
test_tensors2 = []
zero_mask = torch.zeros(50)
zero_mask = torch.reshape(zero_mask, (-1,1))

In [25]:
#Create training sentence tensors
for sen in training_list_80_1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors1.append(temp_stack)
for sen in training_list_80_2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors2.append(temp_stack)

In [26]:
#Create test sentence tensors
for sen in test_list_20_1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors1.append(temp_stack)
for sen in test_list_20_2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors2.append(temp_stack)

In [27]:
#Create Training Master Sets
train_master_set1 = torch.stack(train_tensors1)
train_master_set2 = torch.stack(train_tensors2)

In [28]:
#Create Test Master Sets
test_master_set1 = torch.stack(test_tensors1)
test_master_set2 = torch.stack(test_tensors2)


In [29]:
test_master_set1.shape

torch.Size([113, 1, 10, 50])

In [30]:
#Create Training/Test Master Labels
master_labels = torch.tensor(training_raw_labels).float()/5
train_master_labels = master_labels[0:int(x*0.8)]
test_master_labels = master_labels[int(x*0.8):x]

In [31]:
print(len(train_master_labels))
print(len(test_master_labels))

448
113


In [32]:
#Create Training Set Subsets - Batch size = 37-38
 
train_set1 = []
train_set2 = []
split_train1 = np.array_split(train_tensors1, 12)
split_train2 = np.array_split(train_tensors2, 12)
for i in range(12):
    temp1 = torch.stack(list(split_train1[i]))
    train_set1.append(temp1)
    temp2 = torch.stack(list(split_train2[i]))
    train_set2.append(temp2)

C:\Users\Aaron\Desktop\DLH\Project\Python\penv\lib\site-packages\numpy\core\fromnumeric.py:43: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
C:\Users\Aaron\Desktop\DLH\Project\Python\penv\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), m

In [33]:
type(train_tensors1)

list

In [34]:
#Create Train Label Sets
train_label_sets = []
split_train_labels = np.array_split(train_master_labels, 12)
for i in split_train_labels:
    train_label_sets.append(torch.tensor(i).float())


C:\Users\Aaron\AppData\Local\Temp\ipykernel_17260\740371841.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_label_sets.append(torch.tensor(i).float())


In [36]:
#Import Pytorch Neural Network Packages

import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

In [38]:
#K-max Pooling Code

def kmax_pooling(x, dim, k):
    index = torch.topk(x, k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

In [39]:
#Define Convolutional Neural Network

class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()

        
        self.conv1 = nn.Conv2d(1, 5, 3, padding = 2)
        self.conv2 = nn.Conv2d(5, 25, 3, padding = 2)
        self.fc1 = nn.Linear(4050, 500)
        self.fc2 = nn.Linear(500, 25)
        self.pwd = nn.PairwiseDistance(p=1)

    def forward(self, x1, x2):
        
        x1 = F.relu(self.conv1(x1))
        x1 = kmax_pooling(x1, 2, 3)

        x1 = F.relu(self.conv2(x1))
        x1 = kmax_pooling(x1, 2, 3)
        
        x1 = torch.flatten(x1, 1)
        x1 = self.fc1(x1)
        x1 = self.fc2(x1)
        
        
        x2 = F.relu(self.conv1(x2))
        x2 = kmax_pooling(x2, 2, 3)

        x2 = F.relu(self.conv2(x2))
        x2 = kmax_pooling(x2, 2, 3)    
        
        x2 = torch.flatten(x2, 1)
        x2 = self.fc1(x2)
        x2 = self.fc2(x2)
        
        
        result1 = self.pwd(x1, x2)
        
        result2 = torch.exp(-result1)

        return result2

In [67]:
#Create Model Instance

model = CNN1()

#Define Evaluation Criteria as Mean Square Error Loss

criterion = nn.MSELoss()

#Define Optimization Type as Stochastic Gradient Descent

optimizer = torch.optim.SGD(model.parameters(), lr = np.exp(-1))

In [68]:
#Model Training Code
 
n_epochs = 30
def train_model(model, train_data1, train_data2, training_labels, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim
    
    model.train()

    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for i in range(12):
            train_temp1 = train_data1[i]
            train_temp2 = train_data2[i]
            
            labels_temp = training_labels[i]
            
            y_hat = model(train_temp1, train_temp2)

            
            y_hat = torch.reshape(y_hat, (-1,1))

            labels_temp = torch.reshape(labels_temp, (-1,1))
        
            loss = criterion(y_hat, labels_temp)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
        
            curr_epoch_loss.append(loss.cpu().data.numpy())

        
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")

    return model
 

In [69]:
#Train Model
 
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

model = train_model(model, train_set1, train_set2, train_label_sets)

Epoch 0: curr_epoch_loss=0.41031694412231445
Epoch 1: curr_epoch_loss=0.40950843691825867
Epoch 2: curr_epoch_loss=0.40492773056030273
Epoch 3: curr_epoch_loss=0.3825128972530365
Epoch 4: curr_epoch_loss=0.31125608086586
Epoch 5: curr_epoch_loss=0.28215327858924866
Epoch 6: curr_epoch_loss=0.21388870477676392
Epoch 7: curr_epoch_loss=0.17287154495716095
Epoch 8: curr_epoch_loss=0.16408686339855194
Epoch 9: curr_epoch_loss=0.1634598821401596
Epoch 10: curr_epoch_loss=0.15275293588638306
Epoch 11: curr_epoch_loss=0.1469355970621109
Epoch 12: curr_epoch_loss=0.137239471077919
Epoch 13: curr_epoch_loss=0.13284339010715485
Epoch 14: curr_epoch_loss=0.1275162547826767
Epoch 15: curr_epoch_loss=0.12184181064367294
Epoch 16: curr_epoch_loss=0.11373540759086609
Epoch 17: curr_epoch_loss=0.10692711919546127
Epoch 18: curr_epoch_loss=0.09944432228803635
Epoch 19: curr_epoch_loss=0.09412726014852524
Epoch 20: curr_epoch_loss=0.09160547703504562
Epoch 21: curr_epoch_loss=0.08548129349946976
Epoch 2

In [70]:
#Model Testing Code

def eval_model(model, test_data1, test_data2, target_test):
    """
    :return:
        Y_pred: prediction of model on the dataloder.
            Should be an 2D numpy float array where the second dimension has length 2.
        Y_test: truth labels. Should be an numpy array of ints
    TODO:
        evaluate the model using on the data in the dataloder.
        Add all the prediction and truth to the corresponding list
        Convert Y_pred and Y_test to numpy arrays (of shape (n_data_points, 2))
    """
    model.eval()
    Y_pred = []
    Y_test = []
        
    y_hat = model(test_data1, test_data2)
    y_hat = y_hat.detach().numpy()
    
    Y_pred = y_hat

    Y_test = target_test
        

    return Y_pred, Y_test




In [71]:
import scipy
from scipy.stats import pearsonr

In [72]:
#Test Model and Calcuate Pearson's Correlation Coefficient

y_pred, y_true = eval_model(model, test_master_set1, test_master_set2, test_master_labels)
corr = pearsonr(y_pred, y_true)
print(("Correlation Coefficient: " + str(corr)))

Correlation Coefficient: (0.41109339611095835, 6.096355153011437e-06)
